In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing
import codecs
import re

!python3 -m spacy download en_core_web_md #you will need to install this on first load
import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en_core_web_md')
from IPython.display import HTML
import logging

os.environ['KMP_DUPLICATE_LIB_OK']='True'

W0402 11:51:31.682947 4325553600 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
with codecs.open('inaug_speeches.csv', 'r', encoding='utf-8', errors='ignore') as fdata:
    df = pd.read_csv(fdata)
print(df)

    Unnamed: 0                    Name         Inaugural Address  \
0            4       George Washington   First Inaugural Address   
1            5       George Washington  Second Inaugural Address   
2            6              John Adams         Inaugural Address   
3            7        Thomas Jefferson   First Inaugural Address   
4            8        Thomas Jefferson  Second Inaugural Address   
5            9           James Madison   First Inaugural Address   
6           10           James Madison  Second Inaugural Address   
7           11            James Monroe   First Inaugural Address   
8           12            James Monroe  Second Inaugural Address   
9           13       John Quincy Adams         Inaugural Address   
10          14          Andrew Jackson   First Inaugural Address   
11          15          Andrew Jackson  Second Inaugural Address   
12          16        Martin Van Buren         Inaugural Address   
13          17  William Henry Harrison         I

In [3]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

In [4]:
text = df.iloc[0].text

text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
text = ' '.join(text.split())
doc = nlp(text)

sentences = []
for i in doc.sents:
  if len(i)>1:
    sentences.append(i.string.strip())
    
len(sentences)

23

In [5]:
sentences[0:10]

['fellow-citizens of the senate and of the house of representatives: among the vicissitudes incident to life',
 'no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the th day of the present month.',
 'on the one hand, i was summoned by my country, whose voice i can never hear but with veneration and love, from a retreat which i had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years<u+0097>a retreat which was rendered every day more necessary as well as more dear to me by the addition of habit to inclination, and of frequent interruptions in my health to the gradual waste committed on it by time.',
 'on the other hand, the magnitude and difficulty of the trust to which the voice of my country called me, being sufficient to awaken in the wisest and most experienced of her citizens a distrustful scrutiny into his qualification

In [6]:
embeddings = embed(
    sentences,
    signature="default",
    as_dict=True)["default"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0402 11:52:39.188954 4325553600 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [7]:
%%time
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  x = sess.run(embeddings)

CPU times: user 1min 37s, sys: 6.21 s, total: 1min 43s
Wall time: 47 s


In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
y = pca.fit_transform(x)

from sklearn.manifold import TSNE

y = TSNE(n_components=2).fit_transform(y)

In [9]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


data = [
    go.Scatter(
        x=[i[0] for i in y],
        y=[i[1] for i in y],
        mode='markers',
        text=[i for i in sentences],
    marker=dict(
        size=16,
        color = [len(i) for i in sentences], #set color equal to a variable
        opacity= 0.8,
        colorscale='Viridis',
        showscale=False
    )
    )
]
layout = go.Layout()
layout = dict(
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )
fig = go.Figure(data=data, layout=layout)
file = plot(fig, filename='Sentence encode.html')

from google.colab import files
files.download('Sentence encode.html') 

ModuleNotFoundError: No module named 'google.colab'